In [ ]:
import sys
import os
import numpy as np
from numpy import loadtxt
import cv2

# caffe layers
caffe_root = '/users/vijay.kumar/caffe/'
sys.path.insert(0, caffe_root + 'python')
sys.path.insert(0, '/users/vijay.kumar/tools/liblinear-2.1/python')

import caffe
from caffe import layers as L
from liblinearutil import *
from utils import *

# enable gpu
caffe.set_mode_gpu()

In [ ]:
params = read_params()

In [ ]:
# initilaze caffe transformer
transformer = define_transformer()

# load pose nets and the pose estimator
nets, pose_net = load_models()

In [ ]:
import PIL.Image
from cStringIO import StringIO
import IPython.display

def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [ ]:
# pipa test images 
pipa_test_direc = ## download pipa test set and give full path 
# download from https://people.eecs.berkeley.edu/~nzhang/datasets/pipa_test.tar

# PIPA test set ground truth
pipa_gt_file = '../data/pipa/split_test_original.txt'

In [ ]:
pipa_gt = loadtxt(pipa_ft_file, 
                                  dtype = str)
num_models = len(nets)
num_examples = pipa_gt.shape[0]
pose_features = np.zeros((num_examples, num_models, params['FEATSIZE']));
pose_est_weights = np.zeros((num_examples, 7));
test_labels = np.zeros((num_examples, 1));
test_split = np.zeros((num_examples, 1));

for count in range(num_examples):   
    img_name = pipa_test_direc + str(pipa_gt[count,0]) + '_'  + str(pipa_gt[count,1]) + '.jpg'
    h_box_x = max(int(pipa_gt[count,2]), 0)
    h_box_y = max(int(pipa_gt[count,3]), 0)
    h_box_w = max(int(pipa_gt[count,4]), 0)
    h_box_h = max(int(pipa_gt[count,5]), 0)
    test_labels[count] = int(pipa_gt[count, 6])
    test_split[count] = int(pipa_gt[count, -1])
    
    # read image
    entire_image = cv2.imread(img_name)
    img_h, img_w, img_c = entire_image.shape
    
    # crop head
    head = entire_image[max(0, h_box_y): min(h_box_y + h_box_h, img_h),
                                    max(0, h_box_x): min(h_box_x + h_box_w, img_w), :]

    # UB box
    l = min(h_box_w, h_box_h)
    ub_box_x, ub_box_y = int(h_box_x - 0.5*l), int(h_box_y)
    ub_box_w, ub_box_h = int(2*l), int(4*l)

    # crop UB  
    upper_body = entire_image[max(0, ub_box_y): min(ub_box_y + ub_box_h, img_h),
                                    max(0, ub_box_x): min(ub_box_x + ub_box_w, img_w), :]

    pose_features[count] = get_pose_features(transformer, nets, head, upper_body, num_models, params['FEATSIZE'])      
    pose_est_weights[count] = get_pose_weights(transformer, pose_net, upper_body)
    if count%500==0:        
        print 'Number of images processed:', count

In [ ]:
classifiers = train_linear_classifiers(pose_features[np.where(test_split==0)[0]], 
                                       test_labels[test_split==0], 
                                       num_models,
                                       params)

In [ ]:
# Testing
num_test = np.sum(test_split==1)
predict_scores = None
split_1 = np.where(test_split==1)[0]

split1_test_features = pose_features[split_1]
split1_test_labels = test_labels[split_1]
split1_wts = pose_est_weights[split_1]
    
for i in range(num_test):      
    test_feature = split1_test_features[i]
    test_label = split1_test_labels[i]
    wt = split1_wts[i]          
    pred_sample_sc = pose_aware_identity_prediction_(classifiers, test_feature, test_label, wt,  params, num_models)        
    if i==0:
        predict_scores = np.zeros((num_test, pred_sample_sc.shape[1]))                
    predict_scores[i] = pred_sample_sc 
    
    if i%500==0:
        print 'Number of images predicted:',i  

In [ ]:
# Compute accuracy
Labels = np.squeeze(np.array(classifiers[0][0].get_labels()))
100*np.mean(test_labels[test_split==1] == Labels[np.argmax(predict_scores, axis=1)])